In [16]:
import pandas as pd 
import numpy as np
!pip install xlrd

In [17]:
# The code was removed by Watson Studio for sharing.

,Postalcode,Neighborhood,latitude,longitude
0,110006,Chandni Chowk,28.6505,77.2303
1,110006,Chawri Bazaar,28.6482,77.2270
2,110007,Kamla Nagar,28.6809,77.2046
3,110054,Kashmiri Gate,28.6665,77.2333
4,110005,Karol Bagh,28.6550,77.1888


In [15]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

## Lets get the latitude and longitude of Delhi to draw a map first.

In [18]:
address = 'Delhi, India'

geolocator = Nominatim(user_agent="Delhi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi are 28.6517178, 77.2219388.


## Create a map of Delhi with neighborhoods superimposed on top.

In [19]:
map_Delhi = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postalcode, neighborhood in zip(df['latitude'], df['longitude'], df['Postalcode'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Delhi)  
    
map_Delhi

## Use Define Foursquare Credentials and Version.

In [20]:
# define Foursquare Credentials and Version
CLIENT_ID = 'HUDBNWSLCX5E2WPPJCPHWJC2Z5QXEGGUPLW44DQFAI3XNHGI' # your Foursquare ID
CLIENT_SECRET = 'ZUEL0EWB1F50JH5LMWMTATNKHKBOBLR0L1IZSHERK5MPDEHU' # your Foursquare Secret
VERSION = '20191206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HUDBNWSLCX5E2WPPJCPHWJC2Z5QXEGGUPLW44DQFAI3XNHGI
CLIENT_SECRET:ZUEL0EWB1F50JH5LMWMTATNKHKBOBLR0L1IZSHERK5MPDEHU


## Let's explore the first neighborhood in our dataframe.

In [21]:
df.loc[0, 'Neighborhood']

'Chandni Chowk'

## Get the neighborhood's latitude and longitude values.

In [22]:
neighborhood_latitude = df.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Chandni Chowk are 28.6505, 77.2303.


## Now, let's get the top 100 venues that are in Chandni Chowk within a radius of 500 meters.

In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=HUDBNWSLCX5E2WPPJCPHWJC2Z5QXEGGUPLW44DQFAI3XNHGI&client_secret=ZUEL0EWB1F50JH5LMWMTATNKHKBOBLR0L1IZSHERK5MPDEHU&v=20191206&ll=28.6505,77.2303&radius=500&limit=100'

## Send the GET request and examine the results.

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dea4497edbcad001b217deb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Central Delhi',
  'headerFullLocation': 'Central Delhi, Delhi',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 11,
  'suggestedBounds': {'ne': {'lat': 28.655000004500007,
    'lng': 77.2354182811357},
   'sw': {'lat': 28.645999995499995, 'lng': 77.2251817188643}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b42e3c7f964a520b5da25e3',
       'name': "Karim's | करीम | کریم (Karim's)",
       'location': {'address': '16, Jama Masjid',
        'crossStreet': 'Gali Kababian',
        'lat': 28.64949759295152,
        'lng': 77.2336908547362,
        'lab

## Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Now we are ready to clean the json and structure it into a pandas dataframe.

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Karim's | करीम | کریم (Karim's),Indian Restaurant,28.649498,77.233691
1,Haveli Dharampura,Hotel,28.653247,77.232309
2,Jama Masjid |जामा मस्जिद | جامع مسجد (Jama Ma...,Mosque,28.650136,77.233541
3,Al Jawahar | अल जवाहर | امام جواہر,Indian Restaurant,28.649609,77.233555
4,Chawri Bazaar | चावड़ी बाजार,Hardware Store,28.649771,77.227364


## And how many venues were returned by Foursquare?

In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

11 venues were returned by Foursquare.


## Now we create a function to repeat the same process to all the neighborhoods in Delhi.

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
       # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Now write the code to run the above function on each neighborhood and create a new dataframe called Delhi_venues.

In [29]:
Delhi_venues = getNearbyVenues(df['Neighborhood'],df['latitude'],df['longitude'])

Chandni Chowk
Chawri Bazaar
Kamla Nagar
Kashmiri Gate
Karol Bagh
Connaught Place
INA Colony
Khan Market
Pragati Maidan
Laxmi Nagar
Mayur Vihar
Chhattarpur
New Friends Colony
Green Park
Hauz Khas
Lajpat Nagar
Sarojini Nagar
Siri Fort
South Extension
Palam


In [30]:
Delhi_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chandni Chowk,28.6505,77.2303,Karim's | करीम | کریم (Karim's),28.649498,77.233691,Indian Restaurant
1,Chandni Chowk,28.6505,77.2303,Haveli Dharampura,28.653247,77.232309,Hotel
2,Chandni Chowk,28.6505,77.2303,Jama Masjid |जामा मस्जिद | جامع مسجد (Jama Ma...,28.650136,77.233541,Mosque
3,Chandni Chowk,28.6505,77.2303,Al Jawahar | अल जवाहर | امام جواہر,28.649609,77.233555,Indian Restaurant
4,Chandni Chowk,28.6505,77.2303,Chawri Bazaar | चावड़ी बाजार,28.649771,77.227364,Hardware Store


In [31]:
Delhi_venues.shape

(274, 7)

## Let's check how many venues were returned for each neighborhood.

In [32]:
Delhi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Chandni Chowk,11,11,11,11,11,11
Chawri Bazaar,4,4,4,4,4,4
Chhattarpur,5,5,5,5,5,5
Connaught Place,65,65,65,65,65,65
Green Park,24,24,24,24,24,24
Hauz Khas,6,6,6,6,6,6
INA Colony,6,6,6,6,6,6
Kamla Nagar,23,23,23,23,23,23
Karol Bagh,6,6,6,6,6,6


## Let's find out how many unique categories can be curated from all the returned venues.

In [33]:
print('There are {} uniques categories.'.format(len(Delhi_venues['Venue Category'].unique())))

There are 94 uniques categories.


## Analyze Each Neighborhood.

In [34]:
Delhi_onehot = pd.get_dummies(Delhi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Delhi_onehot['Neighborhood'] = Delhi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Delhi_onehot.columns[-1]] + list(Delhi_onehot.columns[:-1])
Delhi_onehot = Delhi_onehot[fixed_columns]

Delhi_onehot.head()

,Neighborhood,Accessories Store,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beer Garden,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Fast Food Restaurant,Flea Market,Food,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Golf Course,Grocery Store,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Mosque,Movie Theater,Multicuisine Indian Restaurant,Museum,Nightclub,North Indian Restaurant,Other Great Outdoors,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Resort,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Sporting Goods Shop,Stadium,Tea Room,Theater,Train Station,Travel Agency
0,Chandni Chowk,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Chandni Chowk,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Chandni Chowk,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Chandni Chowk,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Chandni Chowk,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
Delhi_onehot.shape

(274, 95)

## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [36]:
Delhi_grouped = Delhi_onehot.groupby('Neighborhood').mean().reset_index()
Delhi_grouped

,Neighborhood,Accessories Store,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beer Garden,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Fast Food Restaurant,Flea Market,Food,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Golf Course,Grocery Store,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Mosque,Movie Theater,Multicuisine Indian Restaurant,Museum,Nightclub,North Indian Restaurant,Other Great Outdoors,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Resort,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Sporting Goods Shop,Stadium,Tea Room,Theater,Train Station,Travel Agency
0,Chandni Chowk,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.090909,0.0,0.000000,0.272727,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,Chawri Bazaar,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,Chhattarpur,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,Connaught Place,0.00,0.000000,0.000000,0.000000,0.015385,0.015385,0.000000,0.015385,0.015385,0.000000,0.076923,0.015385,0.015385,0.000000,0.000000,0.000000,0.015385,0.0000

## Let's print each neighborhood along with the top 5 most common venues.

In [37]:
num_top_venues = 5

for hood in Delhi_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Delhi_grouped[Delhi_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Chandni Chowk----
                           venue  freq
0              Indian Restaurant  0.27
1             Frozen Yogurt Shop  0.09
2                          Hotel  0.09
3  Paper / Office Supplies Store  0.09
4                 Hardware Store  0.09


----Chawri Bazaar----
                  venue  freq
0  Fast Food Restaurant  0.50
1        Hardware Store  0.25
2    Light Rail Station  0.25
3     Accessories Store  0.00
4         Metro Station  0.00


----Chhattarpur----
               venue  freq
0  Indian Restaurant   0.2
1         Food Truck   0.2
2               Café   0.2
3                Pub   0.2
4             Resort   0.2


----Connaught Place----
                venue  freq
0                Café  0.12
1                 Bar  0.08
2   Indian Restaurant  0.08
3         Coffee Shop  0.08
4  Chinese Restaurant  0.06


----Green Park----
               venue  freq
0  Indian Restaurant  0.21
1        Coffee Shop  0.17
2        Pizza Place  0.08
3         Donut Shop  0.08
4     

## Let's put that into a pandas dataframe.

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [39]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Delhi_grouped['Neighborhood']

for ind in np.arange(Delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Chandni Chowk,Indian Restaurant,Hotel,Hardware Store,Paper / Office Supplies Store,Mosque,Frozen Yogurt Shop,Smoke Shop,Light Rail Station,Fast Food Restaurant,Electronics Store
1,Chawri Bazaar,Fast Food Restaurant,Hardware Store,Light Rail Station,Travel Agency,Flea Market,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
2,Chhattarpur,Pub,Café,Indian Restaurant,Food Truck,Resort,Travel Agency,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant
3,Connaught Place,Café,Coffee Shop,Indian Restaurant,Bar,Fast Food Restaurant,Chinese Restaurant,Lounge,South Indian Restaurant,Italian Restaurant,Pub
4,Green Park,Indian Restaurant,Coffee Shop,Pizza Place,Donut Shop,Japanese Restaurant,Korean Restaurant,Lounge,Market,Fast Food Restaurant,Chinese Restaurant
5,Hauz Khas,Coffee Shop,Ice Cream Shop,Dessert Shop,Burger Joint,Event Space,Nightclub,Travel Agency,Fast Food Restaurant,Department Store,Donut Shop
6,INA Colony,Historic Site,Electronics Store,Park,Light Rail Station,Antique Shop,Art Gallery,Hardware Store,Flea Market,Dessert Shop,Donut Shop
7,Kamla Nagar,Fast Food Restaurant,Chinese Restaurant,Indian Restaurant,Donut Shop,Pizza Place,Coffee Shop,Electronics Store,Miscellaneous Shop,Sandwich Place,Café
8,Karol Bagh,Hotel,Snack Place,Pharmacy,Smoke Shop,Cosmetics Shop,Ice Cream Shop,Golf Course,Event Space,Deli / Bodega,Department Store
9,Kashmiri Gate,Market,Ice Cream Shop,Bus Station,Playground,Fast Food Restaurant,Flea Market,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant


## Cluster Neighborhoods.

In [40]:
kclusters = 4


Delhi_grouped_clustering = Delhi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 1, 2, 3, 0, 0, 1, 3, 3],
      dtype=int32)

In [41]:
Cluster_Labels = [1, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 1, 2, 3, 0, 0, 1, 3, 3, 0]

## Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [42]:
Delhi_merged = df

# merge Delhi_grouped with Delhi_data to add latitude/longitude for each neighborhood
Delhi_merged = Delhi_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Delhi_merged # check the last columns!

,Postalcode,Neighborhood,latitude,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,110006,Chandni Chowk,28.6505,77.2303,Indian Restaurant,Hotel,Hardware Store,Paper / Office Supplies Store,Mosque,Frozen Yogurt Shop,Smoke Shop,Light Rail Station,Fast Food Restaurant,Electronics Store
1,110006,Chawri Bazaar,28.6482,77.2270,Fast Food Restaurant,Hardware Store,Light Rail Station,Travel Agency,Flea Market,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
2,110007,Kamla Nagar,28.6809,77.2046,Fast Food Restaurant,Chinese Restaurant,Indian Restaurant,Donut Shop,Pizza Place,Coffee Shop,Electronics Store,Miscellaneous Shop,Sandwich Place,Café
3,110054,Kashmiri Gate,28.6665,77.2333,Market,Ice Cream Shop,Bus Station,Playground,Fast Food Restaurant,Flea Market,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant
4,110005,Karol Bagh,28.6550,77.1888,Hotel,Snack Place,Pharmacy,Smoke Shop,Cosmetics Shop,Ice Cream Shop,Golf Course,Event Space,Deli / Bodega,Department Store
5,110001,Connaught Place,28.6304,77.2177,Café,Coffee Shop,Indian Restaurant,Bar,Fast Food Restaurant,Chinese Restaurant,Lounge,South Indian Restaurant,Italian Restaurant,Pub
6,110023,INA Colony,28.5782,77.2126,Historic Site,Electronics Store,Park,Light Rail Station,Antique Shop,Art Gallery,Hardware Store,Flea Market,Dessert Shop,Donut Shop
7,110003,Khan Market,28.6001,77.2270,Café,Coffee Shop,Bar,Chinese Restaurant,Asian Restaurant,Hotel,Indian Restaurant,Market,Lounge,Bookstore
8,110001,Pragati Maidan,28.6126,77.2434,History Museum,Pool,Historic Site,Café,Museum,Art Museum,Train Station,Grocery Store,Department Store,Dessert Shop
9,110092,Laxmi Nagar,28.6372,77.2756,Accessories Store,Fast Food Restaurant,Pharmacy,Food Truck,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


In [43]:
Delhi_merged.shape

(20, 14)

In [44]:
Delhi_merged['Cluster labels'] = Cluster_Labels
Delhi_merged.head()

,Postalcode,Neighborhood,latitude,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster labels
0,110006,Chandni Chowk,28.6505,77.2303,Indian Restaurant,Hotel,Hardware Store,Paper / Office Supplies Store,Mosque,Frozen Yogurt Shop,Smoke Shop,Light Rail Station,Fast Food Restaurant,Electronics Store,1
1,110006,Chawri Bazaar,28.6482,77.2270,Fast Food Restaurant,Hardware Store,Light Rail Station,Travel Agency,Flea Market,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,2
2,110007,Kamla Nagar,28.6809,77.2046,Fast Food Restaurant,Chinese Restaurant,Indian Restaurant,Donut Shop,Pizza Place,Coffee Shop,Electronics Store,Miscellaneous Shop,Sandwich Place,Café,3
3,110054,Kashmiri Gate,28.6665,77.2333,Market,Ice Cream Shop,Bus Station,Playground,Fast Food Restaurant,Flea Market,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant,3
4,110005,Karol Bagh,28.6550,77.1888,Hotel,Snack Place,Pharmacy,Smoke Shop,Cosmetics Shop,Ice Cream Shop,Golf Course,Event Space,Deli / Bodega,Department Store,3


## Finally, let's visualize the resulting clusters.

In [45]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Delhi_merged['latitude'], Delhi_merged['longitude'], Delhi_merged['Neighborhood'], Delhi_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters.

In [46]:
Delhi_merged.loc[Delhi_merged['Cluster labels'] == 0, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster labels
6,INA Colony,Electronics Store,Park,Light Rail Station,Antique Shop,Art Gallery,Hardware Store,Flea Market,Dessert Shop,Donut Shop,0
14,Hauz Khas,Ice Cream Shop,Dessert Shop,Burger Joint,Event Space,Nightclub,Travel Agency,Fast Food Restaurant,Department Store,Donut Shop,0
15,Lajpat Nagar,Fast Food Restaurant,Market,Movie Theater,Restaurant,Bank,Indie Movie Theater,Café,Sporting Goods Shop,Clothing Store,0
19,Palam,Bar,Men's Store,Museum,IT Services,Grocery Store,Golf Course,Department Store,Dessert Shop,Donut Shop,0


In [47]:
Delhi_merged.loc[Delhi_merged['Cluster labels'] == 1, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster labels
0,Chandni Chowk,Hotel,Hardware Store,Paper / Office Supplies Store,Mosque,Frozen Yogurt Shop,Smoke Shop,Light Rail Station,Fast Food Restaurant,Electronics Store,1
11,Chhattarpur,Café,Indian Restaurant,Food Truck,Resort,Travel Agency,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant,1
16,Sarojini Nagar,Fast Food Restaurant,Pizza Place,Shopping Mall,Restaurant,Travel Agency,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,1


In [48]:
Delhi_merged.loc[Delhi_merged['Cluster labels'] == 2, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster labels
1,Chawri Bazaar,Hardware Store,Light Rail Station,Travel Agency,Flea Market,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,2
12,New Friends Colony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [49]:
Delhi_merged.loc[Delhi_merged['Cluster labels'] == 3, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster labels
2,Kamla Nagar,Chinese Restaurant,Indian Restaurant,Donut Shop,Pizza Place,Coffee Shop,Electronics Store,Miscellaneous Shop,Sandwich Place,Café,3
3,Kashmiri Gate,Ice Cream Shop,Bus Station,Playground,Fast Food Restaurant,Flea Market,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant,3
4,Karol Bagh,Snack Place,Pharmacy,Smoke Shop,Cosmetics Shop,Ice Cream Shop,Golf Course,Event Space,Deli / Bodega,Department Store,3
5,Connaught Place,Coffee Shop,Indian Restaurant,Bar,Fast Food Restaurant,Chinese Restaurant,Lounge,South Indian Restaurant,Italian Restaurant,Pub,3
7,Khan Market,Coffee Shop,Bar,Chinese Restaurant,Asian Restaurant,Hotel,Indian Restaurant,Market,Lounge,Bookstore,3
8,Pragati Maidan,Pool,Historic Site,Café,Museum,Art Museum,Train Station,Grocery Store,Department Store,Dessert Shop,3
9,Laxmi Nagar,Fast Food Restaurant,Pharmacy,Food Truck,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,3
10,Mayur Vihar,Snack Place,Coffee Shop,Department Store,Market,Bakery,Food Truck,Gym / Fitness Center,Gastropub,Furniture / Home Store,3
13,Green Park,Coffee Shop,Pizza Place,Donut Shop,Japanese Restaurant,Korean Restaurant,Lounge,Market,Fast Food Restaurant,Chinese Restaurant,3
17,Siri Fort,Golf Course,Other Great Outdoors,Furniture / Home Store,Australian Restaurant,Indian Restaurant,Chinese Restaurant,Coffee Shop,Stadium,Theater,3
